<a href="https://colab.research.google.com/github/ACE6233/School/blob/main/handson_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hands-on 4
#### The objective of this hands-on is to use YOLOv3 to detect a specified object in a given image.

In [1]:
# -*- coding: utf-8 -*-
"""
YOLO Object Detection on Google Colab

This script demonstrates how to perform object detection using a pre-trained YOLOv3 model
on Google Colab. It allows users to upload an image, specify an object class to detect,
and then visualizes the detection results or informs if the object is not supported or not found.
"""

import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import files
import matplotlib.pyplot as plt

# --- 1. Download YOLOv3 weights and configuration files ---
# These are standard files for YOLOv3. You might need to adjust paths if using a different YOLO version.
print("Downloading YOLOv3 weights and configuration files...")
!wget -nc https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov3.weights
!wget -nc https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget -nc https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
print("Download complete.")

# --- 2. Load YOLO model ---
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO class names (YOLOv3 was trained on COCO dataset)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

print(f"YOLO model loaded. Supported classes: {len(classes)}")

print("\n--- List of 80 objects that can be detected by this YOLO model (COCO dataset) ---")
# Print classes in a readable format, e.g., columns or numbered list
num_columns = 4 # Adjust as needed
for i, obj_class in enumerate(classes):
    print(f"{i+1:3d}. {obj_class:<15}", end="") # 15 for padding
    if (i + 1) % num_columns == 0:
        print() # New line after every 'num_columns' items
if (len(classes) % num_columns != 0): # Print final newline if needed
    print()
print("--------------------------------------------------------------------------------\n")

# --- 3. Function to perform object detection ---
# MODIFIED: Added total_detections_count to the return value
def detect_objects(image_path, target_class=None, confidence_threshold=0.5, nms_threshold=0.4):
    """
    Performs object detection on an image using the loaded YOLO model.

    Args:
        image_path (str): Path to the input image.
        target_class (str, optional): The specific object class to highlight.
                                     If None, all detected objects are shown.
        confidence_threshold (float): Minimum confidence to consider a detection.
        nms_threshold (float): Non-maximum suppression threshold.

    Returns:
        tuple: (processed_image, found_target, total_detections_count)
               processed_image: Image with bounding boxes and labels (or original if no detections).
               found_target: True if the target_class was detected, False otherwise.
               total_detections_count: Total number of objects detected overall.
    """
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return None, False, 0 # Return 0 detections on error

    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    found_target = False

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression to remove redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    total_detections_count = len(indexes) # Count total detections after NMS

    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3)) # Generate unique colors for classes

    if total_detections_count > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[class_ids[i]]

            # Only draw if target_class is None (show all) or if it matches the target_class
            if target_class is None or label.lower() == target_class.lower():
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, label + " " + confidence, (x, y - 10), font, 1, color, 2)
                if target_class is not None and label.lower() == target_class.lower():
                    found_target = True
    return img, found_target, total_detections_count # Return total detections

--2025-09-16 11:39:26--  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov3.weights
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/75388965/e42c2500-9016-11ea-92ba-11df9f79f31b?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-16T12%3A19%3A27Z&rscd=attachment%3B+filename%3Dyolov3.weights&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-16T11%3A18%3A32Z&ske=2025-09-16T12%3A19%3A27Z&sks=b&skv=2018-11-09&sig=z%2Flr28Z0phL04H7VYqsQPL50NwRPmymI3JEQlBs5xfo%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1ODAyMzA2NiwibmJmIjoxNzU4MDIyNzY2LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZ

In [ ]:
# --- 4. Main execution in Colab ---
if __name__ == "__main__":
    print("\n--- YOLO Object Detection Demo ---")
    print("Please upload an image to perform detection.")

    uploaded = files.upload()

    if not uploaded:
        print("No image uploaded. Exiting.")
    else:
        for fn in uploaded.keys():
            image_path = fn
            print(f"Image '{image_path}' uploaded.")

            while True:
                mode_choice = input("\nChoose detection mode:\n1. Detect a specific object\n2. Detect all 80 objects\nType '1', '2', or 'done' to exit: ").strip()

                if mode_choice.lower() == 'done':
                    break
                elif mode_choice == '1':
                    user_input_object = input("Enter the object you want to detect (e.g., 'car', 'dog', 'person'): ").strip()

                    # Check if the requested object is supported by YOLO (i.e., in COCO classes)
                    supported_classes_lower = [c.lower() for c in classes]
                    if user_input_object.lower() not in supported_classes_lower:
                        print(f"'{user_input_object}' is not a supported object class by this YOLO model (COCO dataset). Please refer to the list above.")
                        continue

                    print(f"Attempting to detect '{user_input_object}' in '{image_path}'...")
                    # Call detect_objects with a specific target_class
                    processed_img, target_found, _ = detect_objects(image_path, target_class=user_input_object)

                    if processed_img is not None:
                        plt.figure(figsize=(10, 8))
                        plt.imshow(cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB))
                        plt.axis('off')
                        plt.title(f"Detection Results for '{user_input_object}'")
                        plt.show()

                        if target_found:
                            print(f"Success! '{user_input_object}' detected in the image.")
                        else:
                            print(f"'{user_input_object}' was supported but not detected in the image with current confidence settings.")
                    else:
                        print("Could not process the image for detection.")

                elif mode_choice == '2':
                    print(f"Attempting to detect all 80 objects in '{image_path}'...")
                    # Call detect_objects without a specific target_class
                    processed_img, _, total_detections_count = detect_objects(image_path, target_class=None)

                    if processed_img is not None:
                        plt.figure(figsize=(10, 8))
                        plt.imshow(cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB))
                        plt.axis('off')
                        plt.title("Detection Results (All Supported Objects)")
                        plt.show()

                        if total_detections_count > 0:
                            print(f"Detected {total_detections_count} object(s) of supported classes in the image.")
                        else:
                            print("No objects from the 80 supported classes were detected in the image with current confidence settings.")
                    else:
                        print("Could not process the image for detection.")

                else:
                    print("Invalid choice. Please type '1', '2', or 'done'.")

            print("\nDemo finished.")


--- YOLO Object Detection Demo ---
Please upload an image to perform detection.
